In [ ]:
%matplotlib inline
import mxnet as mx
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn import preprocessing

In [ ]:
batch_size = 100

In [ ]:
model = mx.mod.Module.load('/home/wuyuxiang/quan/insightface/model_params/theBest', 0, context=mx.gpu(0), label_names=None)

In [ ]:
model.bind(data_shapes=[('data', (batch_size, 3, 112, 112))])

In [ ]:
tifs = glob.glob('/home/wuyuxiang/cvpr/caspeal/*.tif')

In [ ]:
pdf = glob.glob('/home/wuyuxiang/cvpr/caspeal/*.pdf')
pdf

In [ ]:
np.prod(cv2.imread(tifs[2]).shape)

In [ ]:
img = cv2.imread(tifs[100])[120:]
img = cv2.resize(img, (112, 112))

In [ ]:
img.shape

In [ ]:
plt.imshow(img)

In [ ]:
np.sum(feature1 * feature2)

In [ ]:
img = (img-127.5)*0.0078125
img = np.transpose(img, (2,0,1))[np.newaxis, :, :, :]
img = mx.nd.array(img) 
db = mx.io.DataBatch(data=(img,), provide_data=[('data', (1, 3, 112, 112))])

In [ ]:
model.forward(db, is_train=False)
feature = model.get_outputs()[0].asnumpy()

In [ ]:
feature = np.zeros((batch_size, 512))
data = np.zeros((batch_size, 3, 112, 112))

In [ ]:
for i in range(10):
    img = cv2.imread(tifs[i])[120:]
    img = cv2.resize(img, (112, 112))
    img = (img-127.5)*0.0078125
    img = np.transpose(img, (2,0,1))[np.newaxis, :, :, :]
    data[i] = img
data = mx.nd.array(data)
db = mx.io.DataBatch(data=(data,), provide_data=[('data', (batch_size, 3, 112, 112))])

In [ ]:
model.forward(db, is_train=False)
feature = model.get_outputs()[0].asnumpy()

In [ ]:
feature.shape

In [ ]:
norm_fea = feature / np.linalg.norm(feature, axis=1, keepdims=True)

In [ ]:
np.dot(feature, feature.T)

## Veri

In [ ]:

data_list = []
for flip in [0, 1]:
    data = mx.nd.empty((len(issame_list) * 2, 3, 112, 112))
    data_list.append(data)
for i in range(len(issame_list) * 2):
    _bin = bins[i]
    img = mx.image.imdecode(_bin)
    img = mx.nd.transpose(img, axes=(2, 0, 1))
    for flip in [0, 1]:
        if flip == 1:
            img = mx.nd.flip(data=img, axis=2)
        data_list[flip][i] = img
    if i % 1000 == 0:
        print('loading bin', i)
print(data_list[0].shape)

In [150]:
data = []

In [151]:
path = '/home/wuyuxiang/faces_ms1m_112x112/' + 'agedb_30.bin'
bins, issame_list = pickle.load(open(path, 'rb'))
print('{} : {}'.format(len(bins), len(issame_list)))

12000 : 6000


In [153]:
path = '/home/wuyuxiang/faces_ms1m_112x112/' + 'lfw.bin'
bins, issame_list = pickle.load(open(path, 'rb'))
print('{} : {}'.format(len(bins), len(issame_list)))

12000 : 6000


In [155]:
path = '/home/wuyuxiang/faces_ms1m_112x112/' + 'cfp_fp.bin'
bins, issame_list = pickle.load(open(path, 'rb'))
print('{} : {}'.format(len(bins), len(issame_list)))

14000 : 7000


In [163]:
type(bins)

list

In [159]:
np.allclose(data[-1], bins[-1])

True

In [ ]:
data = data_list[0][:batch_size]
db = mx.io.DataBatch(data=(data,), provide_data=[('data', (batch_size, 3, 112, 112))])

In [ ]:
model.forward(db, is_train=False)
feature = model.get_outputs()[0].asnumpy()

In [ ]:
feature.shape

In [164]:
a = issame_list

In [165]:
b = [True] * (len(a) * 2)

In [166]:
b[1::2] = a

In [167]:
b[0::2] = a

In [ ]:
embedding1 = embedding[0::2]
embedding2 = embedding[1::2]

In [ ]:
result = np.sum(np.square(embedding1 - embedding2), 1)

In [ ]:
np.less(result, 1.6)

In [ ]:
issame_list[:batch_size]

In [ ]:
import numpy as np
from scipy import interpolate

x = np.arange(0,10)
y = np.exp(-x/3.0)
f = interpolate.interp1d(x, y, kind='cubic')

xnew = np.arange(0,9,0.1)
import matplotlib.pyplot as plt
plt.plot(x,y,'o',xnew,f(xnew),'-')


In [ ]:
f(2)

In [ ]:
real = (issame_list[batch_size*17:batch_size*18-50])

In [ ]:
real[2]=True

In [ ]:
predict  = issame_list[batch_size*17:batch_size*18]

In [ ]:
predict[0] = True

In [ ]:
def calculate_val_far(threshold, dist, actual_issame):
    predict_issame = np.less(dist, threshold)
    true_accept = np.sum(np.logical_and(predict_issame, actual_issame))
    false_accept = np.sum(np.logical_and(predict_issame, np.logical_not(actual_issame)))
    n_same = np.sum(actual_issame)
    n_diff = np.sum(np.logical_not(actual_issame))
    #print(true_accept, false_accept)
    #print(n_same, n_diff)
    val = float(true_accept) / float(n_same + 1e-10)
    far = float(false_accept) / float(n_diff + 1e-10)
    return val, far


In [ ]:
calculate_val_far(0.0, result, real)

In [ ]:
def calculate_val(thresholds, embeddings1, embeddings2, actual_issame, far_target, nrof_folds=2):
    assert(embeddings1.shape[0] == embeddings2.shape[0])
    assert(embeddings1.shape[1] == embeddings2.shape[1])
    nrof_pairs = min(len(actual_issame), embeddings1.shape[0])
    nrof_thresholds = len(thresholds)
    k_fold = LFold(n_splits=nrof_folds, shuffle=False)
    
    val = np.zeros(nrof_folds)
    far = np.zeros(nrof_folds)
    
    diff = np.subtract(embeddings1, embeddings2)
    dist = np.sum(np.square(diff),1)
    indices = np.arange(nrof_pairs)
    
    for fold_idx, (train_set, test_set) in enumerate(k_fold.split(indices)):
        # Find the threshold that gives FAR = far_target
        far_train = np.zeros(nrof_thresholds)
        for threshold_idx, threshold in enumerate(thresholds):
            
            _, far_train[threshold_idx] = calculate_val_far(threshold, dist[train_set], actual_issame[train_set[0]:train_set[0]+len(train_set)])

        if np.max(far_train)>=far_target:
            f = interpolate.interp1d(far_train, thresholds, kind='slinear')
            threshold = f(far_target)
        else:
            threshold = 0.0
    
        val[fold_idx], far[fold_idx] = calculate_val_far(threshold, dist[test_set], actual_issame[test_set[0]:test_set[0]+len(test_set)])
  
    val_mean = np.mean(val)
    far_mean = np.mean(far)
    val_std = np.std(val)
    return val_mean, val_std, far_mean
calculate_val(thresholds, embedding1, embedding2, issame_list[:batch_size/2], 1e-3)

In [ ]:
thresholds = np.arange(0, 4, 0.001)

In [27]:
from torch.utils.data import Dataset, DataLoader
import torch
from numpy import transpose
import mxnet as mx
import pickle
import numpy as np
from tqdm import tqdm

class VerificationDataset(Dataset):
    """
       LFW: 12K
       AgeDB-30: 12K
       CFP-FP: 14K
    """

    def __init__(self, path):
        self.path = [path + 'lfw.bin', path + 'agedb_30.bin', path + 'cfp_fp.bin']
        self.data = []
        self.label = []
        for cur in self.path:
            with open(cur, 'rb') as f:
                data, label = pickle.load(f)
                self.data.extend(data)
                self.label.extend(label)
        self.imgs = len(self.data)
        self.people = len(self.label)

    def __len__(self):
        return self.imgs

    def __getitem__(self, idx):
        img_ = mx.image.imdecode(self.data[idx]).asnumpy()
        img_ = np.transpose(img_, axes=(2, 0, 1))
        return torch.tensor(img_, dtype=torch.float32), torch.tensor([self.label[idx // 2]])


In [35]:
dataset = VerificationDataset('/home/wuyuxiang/faces_ms1m_112x112/')
train_loader = DataLoader(dataset=dataset, batch_size=10, shuffle=True)

In [36]:
for i, (data, target) in enumerate((train_loader)):
    pass

loading data : 20000
loading data : 0
loading data : 15000
loading data : 30000
loading data : 35000
loading data : 5000
loading data : 25000
loading data : 10000


In [38]:
len(train_loader)

3800

In [12]:
# -*- encoding:utf-8 -*- 
from torch.utils.data import Dataset, DataLoader
import torch
import mxnet as mx
import pickle
import numpy as np


class VerificationDataset(Dataset):
    """
       LFW: 12K
       AgeDB-30: 12K
       CFP-FP: 14K
    """

    def __init__(self, path):
        self.path = [path + 'lfw.bin', path + 'agedb_30.bin', path + 'cfp_fp.bin']
        self.data = []
        self.label = []
        for cur in self.path:
            with open(cur, 'rb') as f:
                data, label = pickle.load(f, encoding='bytes')
                self.data.extend(data)
                self.label.extend(label)
        self.imgs = len(self.data)
        self.people = len(self.label)

    def __len__(self):
        return self.imgs

    def __getitem__(self, idx):
        img_ = mx.image.imdecode(self.data[idx]).asnumpy()
        img_ = np.transpose(img_, axes=(2, 0, 1))
        return torch.tensor(img_, dtype=torch.float32), torch.tensor([self.label[idx // 2]])


dataset = VerificationDataset('/home/wuyuxiang/faces_ms1m_112x112/')
train_loader = DataLoader(dataset=dataset, batch_size=10, shuffle=True)
print(True)
for i, (data, target) in enumerate(train_loader):
    pass

True


In [18]:
target.flatten()

tensor([0, 1, 0, 0, 0, 0, 0, 1, 1, 1], dtype=torch.uint8)

In [21]:
True == 2\1

SyntaxError: unexpected character after line continuation character (<ipython-input-21-22e5ed7ba28f>, line 1)

In [25]:
np.array(torch.tensor([1111,111,1,1,1,1]))

array([1111,  111,    1,    1,    1,    1])

In [1]:
import glob
path = '/home/cortex/datashare/ijb/IJB/'

In [2]:
ijb_a = glob.glob(path+'IJB-A/*')

In [3]:
ijb_a

[]

In [4]:
import pandas

ModuleNotFoundError: No module named 'pandas'

In [5]:
import torch

In [6]:
a = torch.load('../../torch_face_params/resnet50.pt')

In [9]:
b = a['model']

In [24]:
b.keys()

TypeError: 'odict_keys' object is not subscriptable

In [13]:
import sys
sys.path.append('../../home/wuyuxiang/quan/FaceRecognition/networks/')

In [20]:
model = ResNet()
model.load_state_dict(a['model'])

RuntimeError: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "res0.weight", "res1.0.fx.0.weight", "res1.0.fx.0.bias", "res1.0.fx.0.running_mean", "res1.0.fx.0.running_var", "res1.0.fx.1.weight", "res1.0.fx.2.weight", "res1.0.fx.3.weight", "res1.0.gx.0.weight", "res1.1.fx.0.weight", "res1.1.fx.0.bias", "res1.1.fx.0.running_mean", "res1.1.fx.0.running_var", "res1.1.fx.1.weight", "res1.1.fx.2.weight", "res1.1.fx.3.weight", "res1.2.fx.0.weight", "res1.2.fx.0.bias", "res1.2.fx.0.running_mean", "res1.2.fx.0.running_var", "res1.2.fx.1.weight", "res1.2.fx.2.weight", "res1.2.fx.3.weight", "res2.0.fx.0.weight", "res2.0.fx.0.bias", "res2.0.fx.0.running_mean", "res2.0.fx.0.running_var", "res2.0.fx.1.weight", "res2.0.fx.2.weight", "res2.0.fx.3.weight", "res2.0.gx.0.weight", "res2.1.fx.0.weight", "res2.1.fx.0.bias", "res2.1.fx.0.running_mean", "res2.1.fx.0.running_var", "res2.1.fx.1.weight", "res2.1.fx.2.weight", "res2.1.fx.3.weight", "res2.2.fx.0.weight", "res2.2.fx.0.bia

In [27]:
mod = InsightFace(85164)

In [29]:
mod.load_state_dict(a['model'])

In [32]:
mod.embedding()

TypeError: forward() missing 1 required positional argument: 'x'